In [1]:
import logging
import os
import datetime
import pandas as pd
import numpy as np
from bokeh.plotting import figure, show
import tkinter as tk
from tkinter import filedialog
import matplotlib.pyplot as plt

In [2]:
def init_logging():
    BASE_DIR = os.path.dirname(os.path.realpath("__file__"))
    LOG_DIR = f'{BASE_DIR}/logs/'
    if not os.path.exists(LOG_DIR):
        os.makedirs(LOG_DIR)
    today = datetime.datetime.now().strftime('%Y%m%d')
    i = 0
    while os.path.exists(f'{LOG_DIR}post_{today}_%s.log' % i):
        i += 1
    LOG_FILE = f'{LOG_DIR}post_{today}_%s.log' % i
    logging.basicConfig(filename=LOG_FILE,level=logging.DEBUG,format='%(asctime)s : %(process)d : %(levelname)s : %(message)s')
    logging.info('Logging started')
    logging.info(f'Log file saved to {LOG_FILE}') 

In [3]:
def open_data_file():
    root = tk.Tk()
    root.withdraw()
    logging.info('Prompting user to select data file')
    #return filedialog.askopenfilename()
    return 'C:/Users/noahr/Documents/GitHub/ntp/Saved_Data/trial_data_MFCC_100_162300.csv'

In [4]:
def load_data_file(f):
    df = pd.read_table(f,header=0,delimiter=',',usecols=range(12),index_col=0)
    logging.info('Converted data file to data frame')
    num_rows = len(df)
    logging.info(f'Dataframe has {num_rows} rows')
    #df = pd.Series.str.split(',', expand=True)
    return df

In [5]:
def add_elapsed_time(df):
    initial_time = df['Time'].values[0]
    elapsed_time = df['Time'].values - initial_time
    df['Elapsed Time'] = elapsed_time
    final_time = elapsed_time[-1]
    logging.info(f'Added elapsed time ({str(final_time)} seconds)')
    return df

In [6]:
def add_sfm_uncertainty(df):
    up = 0.03
    FSR = 240
    u = up*FSR
    df['SFM Uncertainty'] = np.ones(len(df))*u
    return df

In [7]:
init_logging()
f = open_data_file()
logging.info(f'Opened {f}')

df = load_data_file(f)

df = add_elapsed_time(df)

df = add_sfm_uncertainty(df)
df.head()

,Time,Mass Flow,Flow Temperature,Temperature 1,Internal Temperature 1,Temperature 2,Internal Temperature 2,Temperature 3,Internal Temperature 3,Pressure,Valve Position,Elapsed Time,SFM Uncertainty
0,1.638826e+09,155.06,24.53,0.0,0.0,0.0,0.0,0.0,0.0,0.195312,0.25,0.000000,7.2
1,1.638826e+09,155.39,24.50,0.0,0.0,0.0,0.0,0.0,0.0,0.097656,0.50,0.117837,7.2
2,1.638826e+09,155.58,24.52,0.0,0.0,0.0,0.0,0.0,0.0,0.097656,1.00,0.253991,7.2
3,1.638826e+09,154.96,24.51,0.0,0.0,0.0,0.0,0.0,0.0,0.195312,1.25,0.393128,7.2
4,1.638826e+09,155.24,24.59,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.50,0.541845,7.2


In [11]:
p = figure(title="Mass Flow Controller Characterization \n Target: 100 slm",x_axis_label="Elapsed Time (s)", y_axis_label="Mass Flow Rate (slm)")
p.line(df["Elapsed Time"],df["Mass Flow"])
show(p)